# Hardware specifications

*What you will learn:*
- what is a Device and why is a Device needed;
- what are the available devices and where to find them;
- what do a Device and a Channel enforce; 
- tips on how to pick a `Device`;

## The `Device`

As presented in the [introduction to programming a neutral-atom QPU](programming.md#writing-a-pulser-program), the first step to writing a Pulser program is [the selection of a Device](programming.md#pick-a-device). 

The `Device` is an object that stores **all** the physical constraints a quantum program written with Pulser should verify. The `Device` enforces that each operation added to the quantum program (i.e. the `pulser.Sequence`) respects its constraints.  

<div class="alert alert-info">

**Important note**:


The `Device` represents the physics of a neutral-atom QPU but is not the QPU itself. A QPU must be accessed via a `Backend`, which is presented [in this section](./tutorials/backends.nblink).

</div>

### Choosing a `Device`

<center>
<img src="files/decision_diagram_device.png" alt="Decision Diagram to select a Device for the computation" width="600">
</center>


To choose a `Device` for your `Sequence`, the first question you should ask yourself is:

<center>"How close do I want the physical constraints I program with to be to the QPU's?"</center>

**If you want to program with the physical constraints of a QPU**: Each QPU has an associated `Device`, which you can [get from the cloud provider you use to access this QPU](tutorials/backends.nblink#1.2.-Preparation-for-execution-on-QPUBackend).

There are several reasons for which you might want to feel less constrained by the features currently supported by real QPUs. For instance, you might want to design an algorithm for a QPU having better performances (supporting more qubits, longer sequences, ...) or hardware components that have not been installed.

Pulser enables you to define your own devices, but a `Device` object takes as input lots of parameters that have all to be defined. Therefore, for user convenience, `Pulser` provides:

- **Examples of typical physical devices** in `pulser.devices`. Notably, `pulser.AnalogDevice` is an example of a QPU implementing an [Ising Hamiltonian](./programming.md#ising-hamiltonian).

- **The possibility to define a device without some physical constraints** using the `VirtualDevice` class. An example of such a virtual device is the `MockDevice` provided in the `pulser.devices`, which gives full liberty to write a quantum program. `VirtualDevice` is detailed in [an advanced tutorial](./tutorials/virtual_devices.nblink).

<div class="alert alert-warning">

**Note**:

The selection of a device in a Pulser program does not enforce any choice on the [backend](tutorials/backends.nblink). No matter the device you used to program your `Sequence`, you can always submit it to any QPU: if the values of the `Sequence` match the constraints of the `QPU`, it will be executed. 

</div>

<div class="alert alert-info">


**Tip**:

It is possible to change the device with which a `Sequence` was built, by using `Sequence.switch_device`. This is especially useful to check if the values of the `Sequence` match the constraints of the `QPU` prior to submitting to the `QPU`. For instance, you could have built your `Sequence` with an example of a `Device` like `AnalogDevice`, and now want to run it on a QPU, or the specifications of your QPU might have changed between your design and submission.

</div>

### Reading through the `Device`'s specifications

The second question you should ask yourself to choose your `Device` is: 

<center>"Do its constraints allow me to program my `Sequence` ?"</center> 

The device specifications are here to guide your choice. Here are all the parameters in a `Device`:

<div class="alert alert-info">

**Tip**:

It is possible to display some of the specifications of the `Device` with `Device.specs`. See an example with `AnalogDevice.specs` [in the section below](#the-analogdevice).

</div>

### Tips on `Device` selection

The `Device` is going to constrain the next steps of your [quantum program](./programming.md#writing-a-pulser-program):

1) Some parameters are going to constrain [the creation of your Register](./programming.md#create-the-register), and therefore, the [interaction strength in the interaction Hamiltonian](programming.md#interaction-hamiltonian). Some of these parameters are:
    - `dimensions`
    - `max_atom_num`
    - `max_radial_distance`
    - `min_atom_distance`

2) The `rydberg_level` determines the  [Ising interaction coefficient](./programming.md#ising-hamiltonian) $C_6$ of the Ising Hamiltonian. The quantity $\frac{C_6}{\hbar}$ is accessible via the `interaction_coeff` attribute of the `Device`.

3) The `Channels` in the `channel_objects` parameter are going to determine what [Channels are available for the computation](programming.md#pick-the-channels). Knowing what states you want to use in your computation, you can first check that they are among the `Device.supported_states`, then find the bases and their associated channel that enable to use these states using [the conventions page](conventions.md#bases).

4) The `max_sequence_duration` constrains the duration of the [Pulses you can add](programming.md#add-the-pulses), and therefore the Hamiltonian describing the system can at most be defined between 0 and this value.

5) The `max_runs` limits the number of runs a quantum program can be executed on the QPU. See the section on `Backends` to read more about this.

<div class="alert alert-info">

**Note**:

If the Device associated with a QPU has `requires_layout=True`, then you have to define the `Register` from a layout. This adds more constraints to the creation of your `Register`, and is [presented in an advanced tutorial](./tutorials/reg_layouts.nblink).

</div>

## The `Channels`

The third step to writing a Pulser program is [the selection of Channels among the Device](programming.md#pick-a-device).

As a reminder, the selection of a `Channel` defines the [interaction Hamiltonian](programming.md#interaction-hamiltonian) and [the driving Hamiltonian](programming.md#driving-hamiltonian) $H^D$.

$$
H^D(t) / \hbar = \frac{\Omega(t)}{2} e^{-i\phi} |a\rangle\langle b| + \frac{\Omega(t)}{2} e^{i\phi} |b\rangle\langle a| - \delta(t) |b\rangle\langle b|
$$

The `Channels` available for selection are stored in the `channels` property of the `Device`, a dictionnary associating a `channel_id` to each `Channel` in `channel_objects`. For instance, `AnalogDevice` only contains one channel, the `rydberg_global` channel, which can be accessed with `AnalogDevice.channels["rydberg_global"]`. 

### Reading through the `Channel`'s specifications

The `Channel` is defined by:

### Tips on `Channel` selection

The `Channel` is going to determine the computational basis used in the driving Hamiltonian, and what is the Hamiltonian each atom sees:

- The type of the `Channel` defines the [states](conventions.md#bases) that can be addressed by the [driving Hamiltonian](programming.md#driving-hamiltonian) if this channel is picked. All the child classes of `Channel` can be found [here](./apidoc/core.rst#available-channels).
- The addressing of the `Channel` determines what atoms experience the driving Hamiltonian. In general, physical `Channels` have a `Global` addressability, which means that a Pulse added to this channel will implement the same driving Hamiltonian on all the atoms.

The `Channel` also set constraints on the next stage of your quantum program, the addition of Pulses:
- the **duration** of the pulse is constrained by `min_duration` and `max_duration`, as well as `clock_period` (it has to be a multiple of the clock period).
- the **amplitude** is limited by the maximum amplitude `max_amp` and `min_avg_amp`.
- the **detuning** is limited by the maximum absolute detuning `max_abs_det`. It has to be between -`max_abs_det` and `max_abs_det`.

<div class="alert alert-info">

**Note**:

The modulation bandwidth `mod_bandwidth` impacts the duration, the amplitude, the detuning and the phase of the Pulses. It is a more advanced feature explained [in this tutorial](./tutorials/output_mod_eom.nblink).

</div>

## The `AnalogDevice`

In [ ]:
import pulser

print(pulser.AnalogDevice.specs)

The `pulser.AnalogDevice` only supports the $\left|r\right>$ and $\left|g\right>$ states, because it only supports one channel of type "Rydberg" (that can be declared using its name "rydberg_global"). It implements the [Ising Hamiltonian](programming.md#ising-hamiltonian): 

$$\frac{H}{\hbar}(t) = \sum_{k=1}^N \left (\frac{\Omega(t)}{2} e^{-i\phi(t)} |g\rangle\langle r|_k + \frac{\Omega(t)}{2} e^{i\phi(t)} |r\rangle\langle g|_k - \delta(t) |r\rangle\langle r|_k(t) + \sum_{j<k}\frac{C_6}{\hbar R_{kj}^6} \hat{n}_k \hat{n}_j \right)
$$

There:

In [ ]:
from IPython.display import Markdown as md

mylist = [
    r"- The maximum number of atoms $N$ is %i."
    % (pulser.AnalogDevice.max_atom_num),
    r"- The distance between the atoms $R_{ij}$ is at least %i µm and at most %i µm (twice the maximum distance). The distances are defined by placing the atoms in a 2D-plane, constrained by certain positions defined in a [layout](tutorials/reg_layouts.nblink)."
    % (
        pulser.AnalogDevice.min_atom_distance,
        2 * pulser.AnalogDevice.max_radial_distance,
    ),
    r"- $C_6$ is the Ising interaction coefficient: %f."
    % (pulser.AnalogDevice.interaction_coeff),
    r'- The `"rydberg_global"` being a `"Global"` channel, each pulse added to this channel is applied on all the atoms (the quantities $\Omega$, $\delta$, $\phi$ are the same for each atom).',
    r"- The Hamiltonian (and the pulses with their time-dependent quantities $\Omega$, $\delta$, $\phi$) can be defined between 0 and %i (the minimum of the maximum Sequence duration in the Device specifications and the `maximum_duration` of the channel)."
    % (pulser.AnalogDevice.max_sequence_duration),
    r"- The value of $\Omega$ can go between 0 and %f and the value of $\delta$ can go between -%f and %f."
    % (
        pulser.AnalogDevice.channels["rydberg_global"].max_amp,
        pulser.AnalogDevice.channels["rydberg_global"].max_abs_detuning,
        pulser.AnalogDevice.channels["rydberg_global"].max_abs_detuning,
    ),
]
md("\n".join(mylist))